# How to properly prevent overfitting

**Objective**
- Give a validation set to the model
- Use the stopping criterion to prevent the Neural network from overfitting
- Regularize your network

## Data 

First, let's generate some data thanks to the [`make_blob`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html) function that we used yesterday.

❓ **Question** ❓ Generate 2000 samples, with 10 features each. There should be 8 classes of blobs (`centers` argument), wich `cluster_std` equal to 7. Plot some dimensions to check your data.

In [49]:
import numpy as np
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
%matplotlib inline

### YOUR CODE HERE

❓ **Question** ❓ Use the `to_categorical` function from `tensorflow` to convert `y` to `y_cat` which is the categorical representation of `y` with one-hot encoding columns.

In [ ]:
from tensorflow.keras.utils import to_categorical 


## Part I : Proper cross-validation

In a previous challenge, we split the dataset into a train and a test set at the beginning of the notebook. And then, we started to build different models which were trained on the train set but evaluated on the test set.

So, at the end of the day, we used the test set as many times as we evaluated our models and different hyperparameters. We therefore _used_ the test set to select our best model, which is a sort of overfitting.

A first good practice is to avoid using `random_state` or any deterministic separation between your train and test set. In that case, your test set will change everytime you re-run your notebook. But this is far from being sufficient.

To properly compare models, you have to run a proper cross-validation, a 10-fold split for instance. Let's see how to do it properly.

❓ **Question** ❓ First, write a function that outputs a neural network with 3 layers
- a layer with 25 neurons, the `relu` activation function and the appropriate `input_dim`
- a layer with 10 neurons and the `relu` activation function.
- a last layer which is suited to the problem at hand (multiclass classification)

The function should include its compilation, with the `categorical_crossentropy` loss, the `adam` optimizer and the `accuracy` metrics.

In [4]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [5]:
def initialize_model():
    pass  # YOUR CODE HERE

Here, we will do a proper cross validation.

❓ **Question** ❓ Write a loop thanks to the [K-Fold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) function of Scikit-Learn (select 10 splits) to fit your model on the train data, and evaluate it on the test data. Store the result of the evaluation in the `results` variable.

Do not forget to standardize your train data before fitting the neural network.
Also, 150 epochs shoul be sufficient in a first approximation

In [6]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

kf = KFold(n_splits=10)
kf.get_n_splits(X)

results = []

for train_index, test_index in kf.split(X):

    # Split the data into train and test
    ### YOUR CODE HERE
    
    # Initialize the model
    ### YOUR CODE HERE
    
    # Fit the model on the train data
    ### YOUR CODE HERE
    
    # Evaluate the model on the test data and append the result in the `results` variable
    ### YOUR CODE HERE
    
    pass

❓ **Question** ❓ Print the mean accuracy, and its standard deviation

In [10]:
# YOUR CODE HERE

❗ **Remark** ❗ You probably encountered one of the drawback of using a proper cross-validation for a neural network: **it takes a lot of time**. Therefore, for the rest of deep-learning module, we will do **only one split**. But remember that this is not entirely correct and, for real-life applications and problems, you are encouraged to use a proper cross-validation technique.

❗ **Remark** ❗ In general, what practitioners do, is that they split only once, as you did. And once they get to the end of their optimization, they launch a real cross-validation at 6pm, go home and get the final results on the next day.

❓ **Question** ❓ For the rest of the exercise (and of the deep-learning module), split the dataset into train and test with a 70/30% training to test data ratio.



In [11]:
### YOUR CODE HERE

## Part II : Stop the learning before overfitting

Let's first show that if we train the model for too long, it will overfit the training data and will not be good on the test data.

❓ **Question** ❓ To do that, train the same neural network (do not forget to re-initialize it) with `validation_data=(X_test, y_test)` and 500 epochs. Store the history in the `history` variable.

In [13]:
### YOUR CODE HERE

❓ **Question** ❓ Evaluate the model on the test set and print the accuracy

In [15]:
### YOUR CODE HERE

❓ **Question** ❓ Plot the history of the model with the following function : 

In [39]:
def plot_loss_accuracy(history, title=None):
    fig, ax = plt.subplots(1,2, figsize=(13,5))
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('Model loss')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylim((0,3))
    ax[0].legend(['Train', 'Test'], loc='best')
    
    ax[1].plot(history.history['accuracy'])
    ax[1].plot(history.history['val_accuracy'])
    ax[1].set_title('Model Accuracy')
    ax[1].set_ylabel('Accuracy')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Test'], loc='best')
    ax[1].set_ylim((0,1))
    if title:
        fig.suptitle(title)

In [18]:
# YOUR CODE HERE

We clearly see that the number of epochs we choose has a great influence on the final results: 
- If not enough epochs, then the algorithm is not optimal as it has not converged yet. 
- On the other hand, if too many epochs, we overfit the training data and the algorithm does not generalize well on test data.

What we want is basically to stop the algorithm when the test loss is minimal (or the test accuracy is maximal).

Let's introduce the early stopping criterion which is a way to stop the epochs of the algorithm at a interesting epoch. It basically use part of the data to see if the test loss stops from improving. You cannot use the test data to check that, otherwise, it is some sort of data leakage. On the contrary, it uses a subset of the initial training data, called the **validation set**

It basically looks like the following : 

<img src="validation_set.png" alt="Validation set" style="height:350px;"/>

To split this data, we use, in the `fit` function, the `validation_split` keyword which sets the percentage of data from the initial training set used in the validation set. On top of that, we use the `callbacks` keyword to call the early stopping criterion at the end of each epoch. You can check additional information in the [documentation](https://www.tensorflow.org/guide/keras/train_and_evaluate)


❓ **Question** ❓ Launch the following code, plot the history and evaluate it on the test set

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping()

model = initialize_model()

# Fit the model on the train data
history = model.fit(X_train, y_train,
                    validation_split=0.3,
                    epochs=500,
                    batch_size=16, 
                    verbose=0, 
                    callbacks=[es])

In [20]:
# YOUR CODE HERE

❗ **Remark** ❗ The problem, with this type of approach, is that as soon as the loss of the validation set increases, the model stops. However, as neural network convergence is stochastic, it happens that the loss increases before decreasing again. For that reason, the Early Stopping criterion has the `patience` keyword that defines how many epochs without loss decrease you allow.

❓ **Question** ❓ Use the early stopping criterion with a patience of 30 epochs, plot the results and print the accuracy on the test set

In [21]:
### YOUR CODE HERE

❗ **Remark** ❗ The model continues to converge even though it has some loss increase and descrease. The number of patience epochs to select is highly related to the task at hand and there does not exist any general rule. 

❗ **Remark** ❗ In case you select a high patience, you might face the problem that the loss on the test set decrease a lot from the best position. To that end, the early stopping criterion allows you to stop the convergence _and_ restore the weights of the neural network when it had the best score on the validation set, thanks to the `restore_best_weights` that is set to `False` by default.

❓ **Question** ❓ Run the model with a early stopping criterion that enables to restore the best weights of the parameters, plot the loss and accuracy and print the accuracy on the test set

In [23]:
### YOUR CODE HERE

❗ **Remark 1** ❗ You can look at the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) to play with other parameters

❗ **Remark 2** ❗ No need to take a look at the epochs as long as it hit the stopping criterion. So, in the future, you should have a large number of epochs and the early stopping criterion has to stop the epochs. 

## Part III : Batch-size & Epochs

❓ **Question** ❓ Let's run the previous model with different batch sizes (with the early stopping criterion) and plot the results.

In [42]:
# RUN THIS CELL (it can take some time)

es = EarlyStopping(patience=20, restore_best_weights=True)

for batch_size in [1, 4, 32]:
    
    model = initialize_model()

    history = model.fit(X_train, y_train,
                        validation_split=0.3,
                        epochs=500,
                        batch_size=batch_size, 
                        verbose=0, 
                        callbacks=[es])

    results = model.evaluate(X_test, y_test, verbose=0)
    plot_loss_accuracy(history, title=f'------ BATCH SIZE {batch_size} ------\n The accuracy on the test set is of {results[1]:.2f}')

❓ **Question** ❓ Look at the oscillations of the accuracy and loss with respect to the batch size number. Is this coherent with what we saw with the Tensorflow Playground? 

In [26]:
# YOUR ANSWER

❓ **Question** ❓ How many optimizations of the weight are they within one epoch, with respect to the number of data and the batch size? Therefore, is one epoch longer with a large or a small bacth size?

# Part IV: Regularization

In this part of the notebook, we will see how to use regularizers in a neural network. Regularizers are used to prevent overfitting that can happends because very complex networks have many many parameters which tends to overfit the training data.

First, let's initialize a model that has too many parameters for the task (many layers and/or many neurons) such that it overfits the training data  
**To better see the effect, we will not use any early stopping criterion**

In [34]:
# RUN THIS CELL

model = models.Sequential()
model.add(layers.Dense(25, activation='relu', input_dim=10))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(8, activation='softmax'))

# Model compilation
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,  validation_split=0.3,
                    epochs=300, batch_size=batch_size, verbose=0)

results = model.evaluate(X_test, y_test, verbose=0)
print(f'The accuracy on the test set is of {results[1]:.2f}')
plot_loss_accuracy(history)

☝️ In our overparametrized network, some neurons got too specific to given training data, preventing the network from generalizing to new data. This lead to some overfitting. 

For that reason, we will use 
- [`Dropout`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) layers, whose role is to _cancel_ the output of some neurons  during the training part. By doing this at random, it prevents the network from getting too specific to the input data : no any neuron can be too specific to a given input as its output is sometimes cancelled by the dropout layer. Overall, it forces the information that is contain in one input sample to go through multiple neurons instead of only one specific.

- [`Regularizers`](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers), as in linear regression regularization where the weights of the linear regression are constrained by L1, L2 or L1 and L2 norms.

❓ **Question** ❓ Try adding dropout layers and regularization to all your layers of the above neural network and look at the effect on the loss on the test set.

🏁 **Congratulation** 

Don't forget to commit and push your challenge